In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [4]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
#drop irrelevent columns
data = data.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [6]:
#transform the categorical columns
label_encode_gender = LabelEncoder()
data["Gender"] = label_encode_gender.fit_transform(data["Gender"])

In [7]:
#use OneHot encoding for Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encode_geography = OneHotEncoder()
geo_encoder = onehot_encode_geography.fit_transform(data[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
#use OneHot encoding for Geography
onehot_encode_geography.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encode_geography.get_feature_names_out(["Geography"]))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [11]:
data = pd.concat([data.drop("Geography",axis=1),geo_encoder_df],axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [12]:
#Save the encoders
with open("label_encode_gender.pkl","wb") as file:
    pickle.dump(label_encode_gender,file)

with open("onehot_encode_geography.pkl","wb") as file:
    pickle.dump(onehot_encode_geography,file)

In [13]:
#devide dataset into dependent and independent features
x = data.drop("Exited",axis=1)
y = data["Exited"]

x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [14]:
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

ANN

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [17]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
]
)

In [19]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [20]:
# compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
# set up the tensorBoard
logdir = "logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
os.makedirs(logdir, exist_ok=True)
print(f"Created TensorBoard log directory: {logdir}")
print("Directory exists:", os.path.exists(logdir))
tensorflow_callback = TensorBoard(log_dir=logdir,histogram_freq=1)

Created TensorBoard log directory: logs/fit20250221-204113
Directory exists: True


In [51]:
# set up the Earlystopping 
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=4,restore_best_weights=True)

In [52]:
# train the model
history = model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[early_stopping_callback,early_stopping_callback]
)

Epoch 1/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3195 - accuracy: 0.8679 - val_loss: 0.3532 - val_accuracy: 0.8636
Epoch 2/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3228 - accuracy: 0.8663 - val_loss: 0.3466 - val_accuracy: 0.8628
Epoch 3/100
235/235 [==============================] - 1s 2ms/step - loss: 0.3163 - accuracy: 0.8720 - val_loss: 0.3469 - val_accuracy: 0.8600
Epoch 4/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3141 - accuracy: 0.8712 - val_loss: 0.3575 - val_accuracy: 0.8532


In [53]:
model.save('model.h5')

In [ ]:
# load tensorboard extension
%load_ext tensorboard

In [55]:
%tensorboard --logdir logs/fit20250221-204113